In [4]:
from urllib.parse import unquote, quote

# this class is copy from others library that already on the internet
# in order to make py homework execute simpler and easier, I only copy the code that I need this time
class Request:
    def __init__(self, r):
        self.content = r
        self.method = r.split()[0]
        self.path = r.split()[1]
        self.body = r.split('\r\n\r\n', 1)[1]

    def form_body(self):
        return self._parse_parameter(self.body)

    def parse_path(self):
        index = self.path.find('?')
        if index == -1:
            return self.path, {}
        else:
            path, query_string = self.path.split('?', 1)
            query = self._parse_parameter(query_string)
            return path, query

    # the method to access header porperty
    @property
    def headers(self):
        header_content = self.content.split('\r\n\r\n', 1)[0].split('\r\n')[1:]
        result = {}
        for line in header_content:
            k, v = line.split(': ')
            result[quote(k)] = quote(v)
        return result

    # the method only use in this class 
    @staticmethod
    def _parse_parameter(parameters):
        args = parameters.split('&')
        query = {}
        for arg in args:
            k, v = arg.split('=')
            query[k] = unquote(v)
        return query

In [5]:
import os
def response_for_request(request):
    # make the absolute path of the directory
    path = os.getcwd()
    path += request.parse_path()[0]
    print(path)

    # send different response refer to different homework request
    if request.parse_path()[0] == "/redirect.html":
        return "HTTP/1.1 301 Moved Permanently\nContent-Type: text/html\nLocation: /good.html".encode('utf-8')
    elif request.parse_path()[0] == "/good.html":
        return "HTTP/1.1 200 OK\nContent-Type: text/html\n\n<html><head><link href=\"style.css\" rel=\"stylesheet\" type=\"text/css\"></head><body>good</body></html>".encode('utf-8')
    elif request.parse_path()[0] == "/style.css":
        return "HTTP/1.1 200 OK\nContent-Type: text/css\n\nBody {color: red;}".encode('utf-8')
    else:
        return "HTTP/1.1 404 Not Found\n\n<html>body><center><h3>Error 404: File not found</h3><p>Python HTTP Server</p></center></body></html>".encode('utf-8')

In [6]:
import time

count = 0

def serveClient(connection, address):
    global count 
    count += 1
    # input the data piles refer to buffer_size
    stream_r = ''
    buffer_size = 1024
    while True:
        data = connection.recv(1024).decode('utf-8')
        stream_r += data
        # if the data stream finish
        if len(data) < buffer_size:
            break
    
    # if the request is empty than it don't need to response
    if len(stream_r.split()) < 2:
        return

    # parse http header into request object
    request = Request(stream_r)

    # get response generate from my function
    response = response_for_request(request)

    # output the message to log file for debugging
    with open("message.log", "a") as myfile:
        myfile.write("TCP count: " + str(count) + '\n')
        myfile.write(time.ctime() + request.method + request.path + " ")
        myfile.write(str(response))
        myfile.write("\n")

    # send the response
    connection.sendall(response)
    connection.close()

    count -= 1

In [7]:
import socket
import threading

def run_server(host='127.0.0.1', port=8888):
    # Create socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # set binded host and port
        s.bind((host, port))
        while True:
            #listening with at most five client
            s.listen(5)

            # accept the connection
            connection, address = s.accept()
            #print(connection)
            #print(address)

            # thread the serveClient(connection, address) function
            threading.Thread(target = serveClient, args = (connection, address)).start()
            
            

In [8]:
run_server('127.0.0.1', 8888)

/Users/yjack0827/Desktop/http_server_AS1/html/good.html
/Users/yjack0827/Desktop/http_server_AS1/html/style.css
/Users/yjack0827/Desktop/http_server_AS1/html/good.html
/Users/yjack0827/Desktop/http_server_AS1/html/style.css
/Users/yjack0827/Desktop/http_server_AS1/html/good.html
/Users/yjack0827/Desktop/http_server_AS1/html/style.css
/Users/yjack0827/Desktop/http_server_AS1/html/favicon.ico
/Users/yjack0827/Desktop/http_server_AS1/html/redirect.html
/Users/yjack0827/Desktop/http_server_AS1/html/good.html
/Users/yjack0827/Desktop/http_server_AS1/html/style.css


KeyboardInterrupt: 